# Wordle Tweet Dataset


In [4]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import re
import scipy.stats as stats
import sklearn

Cominciamo importando il Dataset e visualizziamo alcune informazioni sulla tabella

In [6]:
wordle = pd.read_csv("WordleMegaData.csv")
wordle.shape
wordle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140196 entries, 0 to 2140195
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   WordleID         int64  
 1   ID               int64  
 2   Created_At       object 
 3   Text             object 
 4   Source           object 
 5   UserID           float64
 6   Username         object 
 7   User_ScreenName  object 
 8   Location         object 
 9   Truncated        bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 149.0+ MB


Andiamo ora a visualizzare le prime occorrenze della tabella

In [201]:
wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,3.681676e+07,Leslie Brown,live_laugh_pray,Haida Gwaii,False
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,4.825910e+08,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.397624e+18,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.255905e+18,Dustin Waters,DustinWaters12,NaN,False
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,2.528937e+08,R. Colin,rollin_fatty,"Richmond, VA",False


## Pulizia dei dati

In [202]:
wordle.isnull().sum()

WordleID                0
ID                      0
Created_At              0
Text                    0
Source                  0
UserID                  0
Username              112
User_ScreenName         0
Location           573373
Truncated               0
dtype: int64

La caratteristica 'ID' rappresenta l'ID del tweet e quindi deve essere unica. Verifichiamo che sia così.

In [203]:
num_unique_elements = wordle['ID'].nunique()
total_elements = len(wordle['ID'])

print("Number of rows where 'ID' is unique:", num_unique_elements)
print("Total number of rows:", total_elements)

Number of rows where 'ID' is unique: 2140187
Total number of rows: 2140196


Sono presenti dei duplicati, li eliminiamo.

In [204]:
wordle = wordle.drop_duplicates(subset=['ID'], keep=False)
wordle.shape

(2140178, 10)

Ora il valore coincide con le occorrenze uniche, andiamo ora a effettura una pulizia del testo, mantenendo solamente le informazioni riguardanti la partita a wordle. 
Inoltre dal testo generiamo una colonna "Attempts" che rappresenta il numero di tentativi effettuati per indovinare la parola o la X nel caso di fallimento.

In [205]:
number_pattern = re.compile(r'(Wordle) (\d{3}) (([1-6]|X)\/6)')

# Function to extract the number of attempts from each row
def extract_attempts(row):
    match = number_pattern.search(row)
    if match:
        return match.group(3).replace('/6', '') 
    else:
        return np.nan

# Create a new column with the extracted numbers
wordle['Attempts'] = wordle['Text'].apply(extract_attempts)

wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated,Attempts
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,3.681676e+07,Leslie Brown,live_laugh_pray,Haida Gwaii,False,3
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,4.825910e+08,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False,4
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.397624e+18,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False,3
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.255905e+18,Dustin Waters,DustinWaters12,NaN,False,3
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,2.528937e+08,R. Colin,rollin_fatty,"Richmond, VA",False,3


Per estrarre ulteriori informazioni possiamo affidarci alle emoji riportate nei tweet, vi sono una serie di rappresentazioni alternative per la stessa informazione. Inoltre possiamo fare una verifica incrociata con il numero di tentativi per verificare la coerenza delle informazioni.

Le informazioni per le varie emoji sono le seguenti:
- ⬛ o ⬜ indicano che la lettera non è presente nella parola
- 🟦 o 🟨 indicano che la lettera è corretta ma non la sua posizione
- 🟩 o 🟧 indicano che la lettera è corretta e anche la sua posizione

*Possiamo sfruttare questa informazione per fare ulteriori analisi e/o test statistici, come per esempio, in media quanta gente indovina dopo una certa quantità di quadratini gialli/blu?*

In [206]:
# Define a regular expression pattern to extract emojis
#DA AGGIUSTARE: VANNO PRESI I QUADRATI IN COERENZA CON 1/6, 2/6 ECC.
# Volendo la si può fare basandosi sul numero di quadratini dato che rimangono solo quelli, (# quadratini / 5) == attempts. 
# tecnicamente i casi in cui la divisione non restituisce un intero sono già stati scartati tramite la regex sottostante

colored_squares_pattern = re.compile(r'((([🟨🟩⬛⬜]{5}\n){1,6})|(([🟧🟦⬛⬜]{5}\n){1,6}))', flags=re.UNICODE)

# Function to extract emojis from a text
def extract_emojis(text):
    matches = colored_squares_pattern.findall(text)
    if not matches:
        return np.nan
    return ''.join(match[0] for match in matches).replace('\n', '')

# Remove non-emoji text from rows in the 'Text' column
wordle['Text'] = wordle['Text'].apply(extract_emojis)

wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated,Attempts
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,🟨🟨⬛⬛⬛🟨⬛🟨⬛🟩,Twitter for iPhone,3.681676e+07,Leslie Brown,live_laugh_pray,Haida Gwaii,False,3
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,⬛⬛⬛⬛🟩⬛⬛⬛⬛🟩⬛🟩⬛⬛🟩🟩🟩🟩🟩🟩,Twitter for iPhone,4.825910e+08,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False,4
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,⬛⬛⬛⬛🟩⬛⬛🟩⬛🟩,Twitter for iPhone,1.397624e+18,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False,3
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,⬛⬛⬛🟨⬛⬛🟨⬛⬛🟩,Twitter for iPhone,1.255905e+18,Dustin Waters,DustinWaters12,NaN,False,3
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,⬛🟨⬛⬛⬛🟩🟩🟩⬛🟩,Twitter for iPhone,2.528937e+08,R. Colin,rollin_fatty,"Richmond, VA",False,3


L'informazione più affidabile su cui basarsi per identificare un utente all'interno del dataset è UserID. L'assenza di valori nulli per questo attributo ci fornisce maggiore sicurezza. Possiamo allora fare a meno delle colonne Username e User_ScreenName che possono contenere identificativi non unici e modificabili da parte dell'utente nel tempo.

Notiamo inoltre che la colonna Location contiene anche valori nulli e valori errati. Come ulteriore verifica, su Kaggle ci viene suggerito che il 27% dei valori è nullo, il 72% non viene associato a una macrocategoria, mentre l'1% dei tweet risulta provenire dagli Stati Uniti. Dopo aver verificato a nostra volta che il numero di valori nulli per questa caratteristica è elevato, eliminiamo la colonna. 

In [207]:
wordle.drop(columns=['Username', 'User_ScreenName', 'Location'], inplace=True)

In [208]:
#droppare le righe dove troncated è true e droppare poi la colonna

In [209]:
#creare colonna vittoria/perdita
wordle['Win'] = ~wordle['Attempts'].isin(['X', np.nan])
wordle['Attempts'] = wordle['Attempts'].replace('X', '6')
wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Truncated,Attempts,Win
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,🟨🟨⬛⬛⬛🟨⬛🟨⬛🟩,Twitter for iPhone,3.681676e+07,False,3,True
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,⬛⬛⬛⬛🟩⬛⬛⬛⬛🟩⬛🟩⬛⬛🟩🟩🟩🟩🟩🟩,Twitter for iPhone,4.825910e+08,False,4,True
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,⬛⬛⬛⬛🟩⬛⬛🟩⬛🟩,Twitter for iPhone,1.397624e+18,False,3,True
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,⬛⬛⬛🟨⬛⬛🟨⬛⬛🟩,Twitter for iPhone,1.255905e+18,False,3,True
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,⬛🟨⬛⬛⬛🟩🟩🟩⬛🟩,Twitter for iPhone,2.528937e+08,False,3,True


In [210]:
wordle.isnull().sum()

WordleID          0
ID                0
Created_At        0
Text          46348
Source            0
UserID            0
Truncated         0
Attempts      63226
Win               0
dtype: int64

In [211]:
nan_rows = wordle[(wordle['Text'].isna()) & (wordle['Attempts'].isna())]
nan_rows_trunc = wordle[(wordle['Text'].isna()) & (wordle['Attempts'].isna()) & (wordle['Truncated']==True)]
# Count the rows
num_nan_rows = len(nan_rows)
num_nan_rows_trunc = len(nan_rows_trunc)

print("Number of rows where both 'Text' and 'Attempts' are NaN:", num_nan_rows)
print("Number of rows where both 'Text' and 'Attempts' are NaN and Truncated is true:", num_nan_rows_trunc)
#confrontare per coerenza text e attempts

#droppare elementi nulli di text, attempts e win

Number of rows where both 'Text' and 'Attempts' are NaN: 22408
Number of rows where both 'Text' and 'Attempts' are NaN and Truncated is true: 15701


Nella fase di pulizia e integrazione dei dati abbiamo perso circa 100000 campioni sui 2140196 campioni totali, quindi circa il 5% della totalità dei campioni. 

In [212]:
#play_per_day is a pandas Series object.
play_per_day = wordle['WordleID'].value_counts()
print(play_per_day.sample(n=10))

#win_per_day is a pandas Series object.
win_per_day = wordle.groupby('WordleID')['Win'].sum()
print(win_per_day.sample(n=10))

#win_percentage is a pandas Series object.
#fill_value=0 indica che viene messo 0 se play_per_day è zero (per evitare una divisione per zero) ma nel nostro caso non si verifica mai una
#divisione per zero (ho già verificato e in win_percentage fill_value=0 non viene mai applicato)
win_percentage = (win_per_day.div(play_per_day, fill_value=0)).mul(100)
print(win_percentage.sample(n=10))

WordleID
326    20000
306    20000
329    20000
264    20000
361    20000
317    20000
303    20000
295    20000
288    20000
399    20000
Name: count, dtype: int64
WordleID
309    19293
265    14957
275    19204
293    18632
277    19099
322    19334
305    19155
288    17485
307    19451
353    19474
Name: Win, dtype: int64
WordleID
376    91.940
353    97.370
339    96.395
316    94.520
269    95.535
255    94.555
309    96.465
303    97.060
346    95.620
308    96.425
dtype: float64


Le percentuali di vincita sembrano tutte concentrate nell'intervallo compreso tra 90% e 100%. Cerchiamo di comprendere se è davvero così.

In [213]:
print(win_percentage.mean())
print(win_percentage.var())
print(win_percentage.mode())
print(win_percentage.median())

94.49374127074205
18.613224452068774
0    96.395
1    96.465
dtype: float64
95.825
